In [1]:
import os
import json
import requests
import pandas as pd

In [2]:
with open('/Users/areevesman/Desktop/google_routes_api.txt', "r") as f:
    apikey = f.read()
    f.close()

In [3]:
def map_station(x):
    return("San+Francisco+fire+department+station+"+str(int(x)))

# replaces spaces with '+' and removes '#'
def map_address(x):
    return(x.replace(' ','+').replace('#','')+"+San+Francisco+CA")

In [4]:
def add_cols(path='../Data/small_pair_files', file='xaa', output_dir="/Users/areevesman//Desktop/processed", apikey=apikey):
    if not os.path.exists(output_dir):
        print(f"Creating {output_dir} to save results.")
        os.mkdir(output_dir)
    pairs = pd.read_csv(os.path.join(path, file),header=None, names=['station_area','address'])
    stations = list(pairs.station_area.map(map_station))
    addresses = list(pairs.address.map(map_address))
    n = pairs.shape[0]
    distances = ['']*n
    durations = ['']*n
    station_areas_OG = pairs.station_area.values
    addresses_OG = pairs.address.values
    unknowns = []
    for j, origin, destination in zip(range(n), stations, addresses):
        if j % 50 == 0:
            print(f"{j+1} of {n}")
        try:
            url = f'https://maps.googleapis.com/maps/api/distancematrix/json?origins={origin}&destinations={destination}&key={apikey}'
            r = requests.get(url)
            js = r.json()
            e = js['rows'][0]['elements']
            distances[j] = e[0]['distance']['value']
            durations[j] = e[0]['duration']['value']  / 60
        except:
            unknowns.append((station_areas_OG[j],addresses_OG[j]))
            distances[j] = 'unknown'
            durations[j] = 'unknown'
    d = {'station_area' : station_areas_OG,
         'address' : addresses_OG,
         'distance' : distances,
         'duration' : durations}
    output_data = pd.DataFrame.from_dict(d)
    output_data.to_csv(f"{output_dir}/{file}_processed.csv", index=False)
    print(f"{len(unknowns)} of {n} not found")
    return unknowns

In [5]:
%%time

add_cols(path='../Data/small_pair_files', file='xaa', apikey=apikey)

1 of 999
51 of 999
101 of 999
151 of 999
201 of 999
251 of 999
301 of 999
351 of 999
401 of 999
451 of 999
501 of 999
551 of 999
601 of 999
651 of 999
701 of 999
751 of 999
801 of 999
851 of 999
901 of 999
951 of 999
3 of 999 not found
CPU times: user 16.3 s, sys: 1.17 s, total: 17.5 s
Wall time: 8min 17s


[(1, '0 Block of ASHBURTON PL'),
 (1, 'EDDY ST/LARKIN ST'),
 (2, '0 Block of STONE ST')]

In [6]:
%%time

x = add_cols(path='../Data/small_pair_files', file='xae', apikey=apikey) # process data, make new file
print(x) # print what was unknown

1 of 1000
51 of 1000
101 of 1000
151 of 1000
201 of 1000
251 of 1000
301 of 1000
351 of 1000
401 of 1000
451 of 1000
501 of 1000
551 of 1000
601 of 1000
651 of 1000
701 of 1000
751 of 1000
801 of 1000
851 of 1000
901 of 1000
951 of 1000
1 of 1000 not found
[(8, 'CALL BOX: 5TH ST/HARRISON ST')]
CPU times: user 16.2 s, sys: 1.22 s, total: 17.4 s
Wall time: 10min 39s
